# PubMed Abstracts Topic Modeling

In [1]:
# %pip install "umap-learn" hdbscan joblib numpy -U

%reload_ext autoreload
%autoreload 2

from pathlib import Path
import numpy as np
import pandas as pd
from python.cogtext.datasets.pubmed import PubMedDataLoader
from python.cogtext.topic_model import TopicModel

In [3]:
# PUBMED = PubMedDataLoader(preprocessed=False, drop_low_occurred_labels=False).load()
# EMBEDDINGS = np.load('models/embeddings/abstracts_UMAPv4.npz')['arr_0']
embeddings = np.load('models/embeddings/abstracts_all-MiniLM-L6-v2.npz')['arr_0']
documents = pd.read_csv('models/cogtext/abstracts_clusters.csv.gz', index_col=0)
reduced_embeddings = np.load('models/embeddings/abstracts_UMAP5d.npz')['arr_0']

In [4]:
model = TopicModel(parametric_umap=False, verbose=True)
clusters, weights = model.fit_transform(embeddings, umap_embeddings=reduced_embeddings)

# drop cluster "-1" and make the rest 1-indexed
documents['cluster'] = np.where(clusters >= 0, clusters + 1, np.nan)

# store
documents.to_csv('models/cogtext/abstracts_clusters.csv.gz', index=True)
np.savez('models/embeddings/abstracts_cluster-weights.npz', weights)

# report cluster frequencies
documents['cluster'].value_counts()

[TopicModel] Reduced embedding dimensions. Now clustering...
[TopicModel] Clustered embeddings. Now computing weights...
[TopicModel] Done!


56.0     5382
161.0    5260
99.0     4230
182.0    4106
444.0    3830
         ... 
147.0     101
106.0     101
201.0     100
570.0     100
82.0      100
Name: cluster, Length: 629, dtype: int64